This project is an Implementaion of Full Text Search with Invereted Index which is stored in mysql database. This Project is implemented using pyspark

<u>The entire project is divided into 6 steps</u>

-Step 0 - Setup Environment and Import Packages

-Step 1 - Create Invereted Index and doc magnitude and store it in a file

-Step 2 - Store it in a mysql

-Step 3 - Lookup Inverted Index and get metrics

-Step 4 - Calculate Cosine Similarity

-Step 5 - Document Ranking


#Step 0 - Setup Environment and Import Packages

In [142]:
!pip install -q wget
import wget
import sys
import os
import string
import pandas as pd
import json
import math
import sqlite3

In [143]:
# utility functions
def wget_pbar_if_not_exists(url, save_path):

  def bar_progress(current, total, width=80):
    progress_message = "Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total)
    sys.stdout.write("\r" + progress_message)
    sys.stdout.flush()

  if not os.path.isfile(f'{save_path}'):
    print('downloading', save_path)
    wget.download(url, save_path, bar=bar_progress)
  else:
    print('file ', save_path, 'already exists')

In [144]:
!apt-get update 
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
wget_pbar_if_not_exists(url='https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz', save_path=f'./spark-2.4.4-bin-hadoop2.7.tgz')

!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
print('4')
import findspark
findspark.init("spark-2.4.4-bin-hadoop2.7")# SPARK_HOME

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

import nltk
nltk.download('stopwords')
nltk.download('punkt')

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

True

#Step 1 - *Create* Invereted Index and doc magnitude and store it in a file




In [ ]:
!unzip input_docs.zip > /dev/null
!ls input_docs/ | wc -l

replace __MACOSX/._input_docs? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

**Create an RDD from a text file**

Each line of the text file becomes an element of the RDD.

In [ ]:
# wholeTextFiles generates an RDD of pair values, 
# where the key is the full path of each file, the value is the content of each file
input = sc.wholeTextFiles("input_docs");

# Now we strip the prefix of filenames and leave only the basename. 
# e.g. 'file:/content/drive/My Drive/Colab Notebooks/data_spark/input_docs/3.html'
# becomes '3.html' 

input2 = input.map(lambda x: (int(os.path.basename(x[0]).split(".")[0]), x[1]))

for x in input2.take(5):
  print(x)

In [ ]:
# Doc to wordlist function
# The output will be a list of tuples such as 
# ("apple", (3,10,10/20)), 
# where 3 is docid, 
# 10 is frequency of "apple" in this doc, 
# 20 is maxf in in this doc.

from bs4 import BeautifulSoup
from collections import Counter
import re

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# for a given doc return a list of tuples of the form (w, (docid, freq, freq/maxfreq))
def dw(docid, htmltext):
  cleantext = BeautifulSoup(htmltext).get_text()
  result = list()
  word_freq = dict()
  max_freq = 0

  for word in re.findall('[a-zA-Z]+', cleantext):
    word = word.lower()
    if word not in stop_words and word not in string.punctuation:
      if word not in word_freq:
        word_freq[word] = 1
      else:
        word_freq[word] += 1
      if word_freq[word] > max_freq:
        max_freq = word_freq[word]

  for word, frequency in word_freq.items():
    word_attrs = (word, (docid, frequency, frequency/max_freq))
    result.append(word_attrs)

  return result

word_docid_freq_tf = input2.flatMap(lambda x: dw(x[0], x[1]))


for r in word_docid_freq_tf.take(5):
  print(r)

In [ ]:
# Now create an RDD as follows 
# (word, [(did1,freq1,tf1), (did2,freq2,tf2), ...])

word_postinglist_freq_tf_grouped = word_docid_freq_tf.groupByKey().mapValues(list)

for r in word_postinglist_freq_tf_grouped.take(5):
  print(r)

In [ ]:
# (word, [(did,freq,tfidf), ...])
# We easily obtain idf as 1/len(postinglist_tf)
# idf = 1/len(postinglist_tf)

def get_tfidf(word, postinglist):
  idf  = 1/len(postinglist)
  new_posting_list = list()
  for element in postinglist:
    tf = element[2]
    tfidf = tf * idf
    result_element = (element[0], element[1], tfidf)
    new_posting_list.append(result_element)

  return (word, new_posting_list)


word_postinglist_freq_tfidf = word_postinglist_freq_tf_grouped.map(lambda x: get_tfidf(x[0], x[1]))


for r in word_postinglist_freq_tfidf.take(5):
  print(r)

In [ ]:
# Now, we would like to obtain the magnitude of each doc.
# First, produce (did, (freq,tfidf)) for each word of doc did; 
# We do don't need the word itself, just its (freq,tfidf). 
# Then, do reduceByKey on these tuples and obtain maxfreq and 
# magnitude (squared) for each document. 


import itertools
import functools
import operator

from collections import Iterable


def flatten(items):
    """Yield items from any nested iterable; see Reference."""
    for x in items:
        if isinstance(x, Iterable) and not isinstance(x, (str, bytes)):
            for sub_x in flatten(x):
                yield sub_x
        else:
            yield x

def tfidfsq(word, posting_list):
  result = list()
  for element in posting_list:
    new_element = (element[0], (element[1], element[2]))
    result.append(new_element)
  return result


def find_mag(docid, max_freq_mag_list):
  max_freq = max_freq_mag_list[0]
  mag_list = max_freq_mag_list[1]

  mag_squared = 0
  for ele in mag_list:
    mag_squared += ele*ele
  return (docid, (max_freq, mag_squared))

did_freq_tfidfsq_rdd = word_postinglist_freq_tfidf.flatMap(lambda x: tfidfsq(x[0], x[1]))


doc_maxf_mag = did_freq_tfidfsq_rdd.reduceByKey(lambda a,b: (max(a[0], b[0]), list(flatten([a[1], b[1]]))))

doc_maxf_mag = doc_maxf_mag.map(lambda a: find_mag(a[0], a[1]))

for r in doc_maxf_mag.take(5):
  print(r)

In [ ]:
!rm -rf inv_idx
word_postinglist_freq_tfidf.saveAsTextFile("inv_idx");

In [ ]:
!rm -rf doc_mag
doc_maxf_mag.saveAsTextFile("doc_mag");

In [ ]:
!ls -lrt inv_idx
!head inv_idx/part-00001
!wc -l inv_idx/part-00000
!wc -l inv_idx/part-00001
!cat inv_idx/part-00000 inv_idx/part-00001 > /content/drive/MyDrive/inv_idx.txt
!wc -l /content/drive/MyDrive/inv_idx.txt

In [ ]:
!ls -lrt doc_mag
!head doc_mag/part-00000
!wc -l doc_mag/part-00000
!wc -l doc_mag/part-00001
!cat doc_mag/part-00000 doc_mag/part-00001 > /content/drive/MyDrive/doc_mag.txt
!wc -l /content/drive/MyDrive/doc_mag.txt



#Step 2: Store it in a mysql


In [ ]:
sql_stmt_create_postings = "CREATE TABLE IF NOT EXISTS postings ( word VARCHAR(100) PRIMARY KEY, postinglist_freq_tfidf TEXT );"
sql_stmt_create_docmag = "CREATE TABLE IF NOT EXISTS docmag ( docid INT PRIMARY KEY, maxf INT, mag FLOAT );"

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Exception as e:
        print(e)


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Exception as e:
        print(e)

    return conn

os.remove('inverted_index_db.db') if os.path.exists('inverted_index_db.db') else None
my_conn = create_connection('inverted_index_db.db')
create_table(my_conn, sql_stmt_create_postings)
create_table(my_conn, sql_stmt_create_docmag)

#convert doc_mag.txt to dataframe
df_doc_mag = pd.read_csv('/content/drive/MyDrive/doc_mag.txt', delimiter = "\t", names=['raw_text'], index_col=False)
df_doc_mag['raw_text'] = df_doc_mag['raw_text'].apply(lambda x: x.replace('(', '').replace(')', ''))
df_doc_mag[['docid', 'maxf', 'mag']] = df_doc_mag['raw_text'].str.split(',', 2, expand=True)
del df_doc_mag['raw_text']
df_doc_mag = df_doc_mag.set_index('docid')

#convert inv_idx.txt to dataframe
df_doc_inv_idx = pd.read_csv('/content/drive/MyDrive/inv_idx.txt', delimiter = "\t", names=['raw_text'], index_col=False)
df_doc_inv_idx['raw_text'] = df_doc_inv_idx['raw_text'].apply(lambda x: x[1:-1])
df_doc_inv_idx[['word', 'postinglist_freq_tfidf']] = df_doc_inv_idx['raw_text'].str.split(',', 1, expand=True)

df_doc_inv_idx['word'] = df_doc_inv_idx['word'].apply(lambda x: x.replace("'", '').replace('"', ''))

del df_doc_inv_idx['raw_text']
df_doc_inv_idx = df_doc_inv_idx.set_index('word')


#dataframe to sql
df_doc_mag.to_sql('docmag', my_conn, if_exists='append')
df_doc_inv_idx.to_sql('postings', my_conn, if_exists='append')

###verify if if the sql database was correctly populated

In [ ]:
df_doc_mag = pd.read_sql('select * from docmag', my_conn, index_col='docid')
print('df_doc_mag')
print(df_doc_mag.dtypes)
print(df_doc_mag.head())


df_doc_inv_idx = pd.read_sql('select * from postings', my_conn, index_col='word')
print('\n\ndf_doc_inv_idx')
print(df_doc_inv_idx.dtypes)
print(df_doc_inv_idx.head())

# Step 3: Lookup Inverted Index and get metrics

In [ ]:

def find_inv_idx(words):
  quoted_words = [f"'{word}'" for word in words]
  sql_tatement = f"SELECT * from postings where word IN ({ ', '.join(quoted_words) })"
  df_posting_list = pd.read_sql(sql_tatement, my_conn, index_col='word')

  data = []

  for i in df_posting_list.itertuples():
    row_list = re.findall(r'\(.*?\)', i[1].strip())
    for row in row_list:
      row = row[1:-1].split(',')
      data.append([i[0], row[0], row[1], row[2]])

  #unstack posting
  df_posting_list_unstacked = pd.DataFrame(data=data, columns=['word', 'docid', 'freq', 'tfidf'])
  df_posting_list_unstacked["docid"] = pd.to_numeric(df_posting_list_unstacked["docid"])
  df_posting_list_unstacked["freq"] = pd.to_numeric(df_posting_list_unstacked["freq"])
  df_posting_list_unstacked["tfidf"] = pd.to_numeric(df_posting_list_unstacked["tfidf"])

  final_df = df_posting_list_unstacked.join(df_doc_mag, on='docid', how='left', lsuffix="_left")
  del final_df['mag']

  return final_df

search_words = ['business', 'relationships']
inv_index_metrics = find_inv_idx(search_words)
print(inv_index_metrics)

In [ ]:
def get_query_metrics(words, search_inverted_index_metrics):
  word_freq = dict()
  word_idf = dict()
  for word in words:
    if word not in word_freq:
      word_freq[word] = 1
    else:
      word_freq[word] += 1
    
  max_freq = 0
  for freq in word_freq.values():
    if max_freq < freq:
      max_freq = freq

  for word, df in search_inverted_index_metrics.groupby('word'):
    word_idf[word] = 1/df.shape[0]
  
  for word in word_freq:
    if word not in word_idf:
      word_idf[word] = 0

  data = list()
  for word in word_freq:
    data.append([word, word_freq[word], max_freq, word_idf[word]])
  
  df_query_metrics = pd.DataFrame(data=data, columns=['word', 'freq', 'maxf', 'idf'])
  return df_query_metrics

search_words = ['business', 'relationships']
search_query_metrics = get_query_metrics(search_words, search_inverted_index_metrics)
print(search_query_metrics)

# Step 4 Calculate Cosine Similarity

In [ ]:
def cosine_similarity(query, document):
  query = query.set_index('word').T.to_dict()
  document = document.set_index('word').T.to_dict()

  numerator = 0

  #calculate query[word]['tfidf']
  for word in query:
      query[word]['tfidf'] = ((query[word]['freq']/query[word]['maxf']) * query[word]['idf'])

  #calculate numerator
  for word in query:
    if word in document:
      numerator += query[word]['tfidf'] * document[word]['tfidf']

  #calculate denominator
  mag_query = 0
  for word in query:
    mag_query += query[word]['tfidf'] * query[word]['tfidf']

  mag_document = 0
  for word in document:
    mag_document += document[word]['tfidf'] * document[word]['tfidf']

  denominator =  math.sqrt(mag_query) * math.sqrt(mag_document)

 
  cosine_similarity = numerator/denominator
  return cosine_similarity

cosine_similarity(search_query_metrics, inv_index_metrics.loc[inv_index_metrics['docid']==1])

#Step 5 Document Ranking

In [ ]:
def rank_documents(df_query_metrics, inv_index_metrics, show_top=20):
  doc_score = dict()
  for docid, df_doc_metrics in inv_index_metrics.groupby('docid'):
    doc_score[docid] = cosine_similarity(df_query_metrics, df_doc_metrics)
  
  if not doc_score:
    print('No matches found')
  res_count = 0
  for w in sorted(doc_score, key=doc_score.get, reverse=True):
      res_count += 1
      if res_count > show_top:
        break
      print('doc {:>8}   {:<20}'.format(w, doc_score[w]))
      

rank_documents(search_query_metrics, inv_index_metrics)

# User Interface

Enter words here

In [ ]:
#@title Enter Search Query Here

#type your search query here
input_words = 'business relationships' #@param (type:"string")
# input_words = 'interest payments' #@param (type:"string")
# input_words = 'would' #@param (type:"string")

#remove stop words
search_words = [word.lower().strip() for word in input_words.split() if word not in stop_words and word not in string.punctuation]

search_inverted_index_metrics = find_inv_idx(search_words)
search_query_metrics = get_query_metrics(search_words, search_inverted_index_metrics)

rank_documents(search_query_metrics, search_inverted_index_metrics)

below output shows some meta data to understand document scoring

In [ ]:
print('META DATA')
print('\n\n')
print('inverted_index_metrics')
print(search_inverted_index_metrics)
print('\n\n')
print('query_metrics')
print(search_query_metrics)
print('\n\n')
print('cosine similarity with doc 3')
cosine_similarity(search_query_metrics, search_inverted_index_metrics.loc[search_inverted_index_metrics['docid']==3])